<a href="https://colab.research.google.com/github/alexD1990/Data_engineering/blob/main/financial_data_extract_transform_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1 Collecting data from storage account (Extract)
from google.colab import drive
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

def mount_google_drive(): # Function to mount Google Drive if not mounted, otherwise unmount and remount
    drive.flush_and_unmount()
    drive.mount('/content/drive')

mount_google_drive() # Mount Google Drive
file_path = "filepath_to_your_csv.csv" # Path to the CSV file in your Storage account

# Read the CSV file into a DataFrame
df_storage_account = pd.read_csv(file_path)

df_storage_account.columns = df_storage_account.columns.str.strip() # Clean column names for whitespace
df_storage_account_ticker = df_storage_account['Ticker'] # Create a list of tickers

Mounted at /content/drive


In [ ]:
# Step 2 Collect data from yahoo finance API (Extract)
import yfinance as yf
import pandas as pd


def get_financial_data(tickers):
  """
  This function retrieves income statement data and closing prices for a list of ticker symbols.

  Args:
      tickers (list): A list of ticker symbols of the companies.

  Returns:
      dict: A dictionary where keys are ticker symbols and values are pandas DataFrames containing financial data.
  """

  # Initialize dictionary to store dataframes
  data_by_ticker = {}

  # Loop through each ticker
  for ticker in tickers:
    try:
      # Get income statement data
      company = yf.Ticker(ticker)
      df = company.income_stmt
      df_transposed = df.T
      new_df = df_transposed[['Total Revenue', 'Diluted EPS', 'Net Income']]
      new_df.index = new_df.index.strftime('%Y')  # Change index to year

      # Download closing prices for the past 4 years
      closing_prices = {}
      for year in range(2020, 2024):
        start_date = f"{year}-06-29"
        end_date = f"{year}-06-30"
        year_data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
        closing_prices[year] = year_data['Adj Close'].iloc[0]

      # Create dataframe for closing prices
      adj_close_df = pd.DataFrame(closing_prices.values(), index=closing_prices.keys(), columns=['Adj Close'])
      adj_close_df.index = adj_close_df.index.astype(str)  # Ensure year is string

      # Combine DataFrames
      combined_df = pd.concat([new_df, adj_close_df], axis=1)

      # Change data types
      data_type_dict = {
        'Total Revenue': 'int64',
        'Diluted EPS': 'float64',
        'Net Income': 'int64',
        'Adj Close': 'float64'
      }
      combined_df = combined_df.astype(data_type_dict)

      # Add dataframe to dictionary with ticker as key
      data_by_ticker[ticker] = combined_df
    except Exception as e:
      print(f"Error downloading data for {ticker}: {e}")

  return data_by_ticker

# Example usage
tickers = df_storage_account_ticker # Replace with your list of tickers
data_by_ticker = get_financial_data(tickers)


# You can now work with individual dataframes for each ticker

data_by_ticker


In [ ]:
# Step 3 transform and make calculation on extracted data (Transform)

def calculate_cagr(dataframe, column_name):
  """Calculates the CAGR for a specific column in a DataFrame.

  Args:
      dataframe (pandas.DataFrame): The DataFrame containing the data.
      column_name (str): The name of the column for which CAGR is calculated.

  Returns:
      float: The CAGR for the specified column.
  """

  # Get the opening and closing values for the column
  current_value = dataframe[column_name].iloc[0]
  past_value = dataframe[column_name].iloc[-1]

  # Calculate the number of years (assuming annual data)
  number_of_years = len(dataframe)

  # Ensure non-zero opening value to avoid division by zero
  if current_value == 0:
    return None  # Handle cases where opening value is zero

  # Calculate CAGR using the formula
  cagr = ((current_value / past_value) ** (1 / number_of_years)) - 1

  return cagr   # Express CAGR as a percentage

# Empty list to store estimated dataframes for each ticker
estimated_dataframes = []

# Loop through each ticker (DataFrame) in the dictionary
for ticker, dataframe in data_by_ticker.items():

    # Calculate PE ratio and profit margin
    dataframe['PE'] = dataframe['Adj Close'] / dataframe['Diluted EPS']
    dataframe['Profit margin'] = dataframe['Net Income'] / dataframe['Total Revenue']

    # Calculate CAGR for Total Revenue and Diluted EPS
    revenue_cagr = calculate_cagr(dataframe, 'Total Revenue')
    eps_cagr = calculate_cagr(dataframe, 'Diluted EPS')

    # Get first year's revenue and EPS for future estimates
    first_year_revenue = dataframe['Total Revenue'].iloc[0]
    first_year_eps = dataframe['Diluted EPS'].iloc[0]

    # Calculate mean PE ratio for the ticker
    mean_pe = statistics.mean(dataframe['PE'])

    # Create a dictionary to store estimates for this ticker
    ticker_estimates = {
        'Ticker': ticker,
        'Revenue CAGR': revenue_cagr,
        'EPS CAGR': eps_cagr,
        '2023 Revenue': first_year_revenue,
        '2023 EPS': first_year_eps,
        'Mean PE': mean_pe,
    }

    # Extract last day's closing price using Yahoo Finance
    company = yf.Ticker(ticker)
    historical_data = company.history(period="1d")
    last_close_price = historical_data['Close'][0]

    # Calculate future estimates and CAGR for stock price
    ticker_estimates['2027 Revenue Est'] = first_year_revenue * (1 + revenue_cagr) ** len(dataframe)
    ticker_estimates['2027 EPS Est'] = first_year_eps * (1 + eps_cagr) ** len(dataframe)
    ticker_estimates['2027 Stock Price Est'] = ticker_estimates['2027 EPS Est'] * mean_pe
    ticker_estimates['Current Stock Price'] = last_close_price
    ticker_estimates['Stock Price CAGR'] = ((ticker_estimates['2027 Stock Price Est'] / last_close_price) ** (1 / len(dataframe))) - 1

    # Create a DataFrame from the estimates dictionary
    ticker_estimates_df = pd.DataFrame([ticker_estimates])

    # Append the DataFrame for this ticker to the list
    estimated_dataframes.append(ticker_estimates_df)

# Combine all DataFrames into a single DataFrame
all_estimates_df = pd.concat(estimated_dataframes)

# Print the resulting DataFrame containing estimates for all tickers
all_estimates_df



In [137]:
# Step 4 merge stock info from storage account with new dataframe (transform and clean)

final_df = df_storage_account.merge(all_estimates_df, on='Ticker')
final_df = final_df.drop('Morningstar Rating for Stocks', axis=1)
new_column_order = ['Ticker',
                    'Name',
                    'Sector',
                    'Industry',
                    'Stock Style Box',
                    'Market Cap  ($)',
                    'Economic Moat',
                    'Capital Allocation',
                    '2023 Revenue',
                    '2027 Revenue Est',
                    'Revenue CAGR',
                    '2023 EPS',
                    '2027 EPS Est',
                    'EPS CAGR',
                    'Mean PE',
                    'Current Stock Price',
                    '2027 Stock Price Est',
                    'Stock Price CAGR']
final_df = final_df[new_column_order]

In [138]:
# Step 5 Loading cleaned and transformed dataframe to storage account as csv (Loading)

filename = 'stock_list.csv'
filepath = 'your_filepath_to_your_storage_account' + filename #Filepath to storage environment

# Save the DataFrame to a CSV file
final_df.to_csv(filepath, index=False)

In [140]:
final_df.head() # Preview of final dataframe

,Ticker,Name,Sector,Industry,Stock Style Box,Market Cap ($),Economic Moat,Capital Allocation,2023 Revenue,2027 Revenue Est,Revenue CAGR,2023 EPS,2027 EPS Est,EPS CAGR,Mean PE,Current Stock Price,2027 Stock Price Est,Stock Price CAGR
0,MMM,3M Co,Industrials,Conglomerates,Large Value,51168966000,Narrow,Standard,32681000000,3.318567e+10,0.003838,-12.63,NaN,NaN,8.822812,104.070000,NaN,NaN
1,ABBNY,ABB Ltd ADR,Industrials,Electrical Equipment & Parts,Large Growth,84351666344,Wide,Standard,32235000000,3.976028e+10,0.053854,2.01,1.662593,-0.046331,15.512308,47.860001,25.790649,-0.143214
2,AMCX,AMC Networks Inc Class A,Communication Services,Entertainment,Small Value,541469967,Narrow,Standard,2711877000,2.612573e+09,-0.009283,4.90,5.174569,0.013724,47.601122,12.670000,246.315288,1.099804
3,AME,AMETEK Inc,Industrials,Specialty Industrial Machinery,Mid Core,41166460467,Wide Q,Exemplary,6596950000,9.585787e+09,0.097921,5.67,8.527560,0.107415,25.863735,181.529999,220.554540,0.049885
4,AMN,AMN Healthcare Services Inc,Healthcare,Medical Care Facilities,Small Value,2078186364,Narrow,Standard,3789254000,5.998397e+09,0.121684,5.36,19.411892,0.379513,19.129649,59.419998,371.342671,0.581104
